# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 11 2023 2:22PM,254822,10,Clear-9019345,"ClearSpec, LLC",Released
1,Jan 11 2023 2:09PM,254821,21,OTM100960130,"NBTY Global, Inc.",Released
2,Jan 11 2023 2:05PM,254819,21,OTM100959983,"NBTY Global, Inc.",Released
3,Jan 11 2023 1:56PM,254815,10,0086294155,ISDIN Corporation,Released
4,Jan 11 2023 1:56PM,254815,10,0086294156,ISDIN Corporation,Released
5,Jan 11 2023 1:56PM,254815,10,0086294157,ISDIN Corporation,Released
6,Jan 11 2023 1:56PM,254815,10,0086294161,ISDIN Corporation,Released
7,Jan 11 2023 1:55PM,254814,15,30013117,"Alliance Pharma, Inc.",Released
8,Jan 11 2023 1:55PM,254814,15,30013118,"Alliance Pharma, Inc.",Released
9,Jan 11 2023 1:55PM,254814,15,30013119,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,254814,Released,53
46,254815,Released,4
47,254819,Released,1
48,254821,Released,1
49,254822,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254814,NaN,NaN,53.0
254815,NaN,NaN,4.0
254819,NaN,NaN,1.0
254821,NaN,NaN,1.0
254822,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254697,0.0,0.0,1.0
254727,1.0,9.0,19.0
254729,0.0,0.0,13.0
254736,2.0,5.0,3.0
254737,3.0,6.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254697,0,0,1
254727,1,9,19
254729,0,0,13
254736,2,5,3
254737,3,6,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254697,0,0,1
1,254727,1,9,19
2,254729,0,0,13
3,254736,2,5,3
4,254737,3,6,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254697,,,1
1,254727,1,9,19
2,254729,,,13
3,254736,2,5,3
4,254737,3,6,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 11 2023 2:22PM,254822,10,"ClearSpec, LLC"
1,Jan 11 2023 2:09PM,254821,21,"NBTY Global, Inc."
2,Jan 11 2023 2:05PM,254819,21,"NBTY Global, Inc."
3,Jan 11 2023 1:56PM,254815,10,ISDIN Corporation
7,Jan 11 2023 1:55PM,254814,15,"Alliance Pharma, Inc."
60,Jan 11 2023 1:54PM,254812,10,Methapharm-G
61,Jan 11 2023 1:50PM,254813,21,"NBTY Global, Inc."
62,Jan 11 2023 1:44PM,254811,21,"NBTY Global, Inc."
63,Jan 11 2023 1:37PM,254810,21,"NBTY Global, Inc."
64,Jan 11 2023 1:28PM,254806,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 11 2023 2:22PM,254822,10,"ClearSpec, LLC",,,1
1,Jan 11 2023 2:09PM,254821,21,"NBTY Global, Inc.",,,1
2,Jan 11 2023 2:05PM,254819,21,"NBTY Global, Inc.",,,1
3,Jan 11 2023 1:56PM,254815,10,ISDIN Corporation,,,4
4,Jan 11 2023 1:55PM,254814,15,"Alliance Pharma, Inc.",,,53
5,Jan 11 2023 1:54PM,254812,10,Methapharm-G,,,1
6,Jan 11 2023 1:50PM,254813,21,"NBTY Global, Inc.",,,1
7,Jan 11 2023 1:44PM,254811,21,"NBTY Global, Inc.",,,1
8,Jan 11 2023 1:37PM,254810,21,"NBTY Global, Inc.",,,1
9,Jan 11 2023 1:28PM,254806,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 2:22PM,254822,10,"ClearSpec, LLC",1,,
1,Jan 11 2023 2:09PM,254821,21,"NBTY Global, Inc.",1,,
2,Jan 11 2023 2:05PM,254819,21,"NBTY Global, Inc.",1,,
3,Jan 11 2023 1:56PM,254815,10,ISDIN Corporation,4,,
4,Jan 11 2023 1:55PM,254814,15,"Alliance Pharma, Inc.",53,,
5,Jan 11 2023 1:54PM,254812,10,Methapharm-G,1,,
6,Jan 11 2023 1:50PM,254813,21,"NBTY Global, Inc.",1,,
7,Jan 11 2023 1:44PM,254811,21,"NBTY Global, Inc.",1,,
8,Jan 11 2023 1:37PM,254810,21,"NBTY Global, Inc.",1,,
9,Jan 11 2023 1:28PM,254806,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 2:22PM,254822,10,"ClearSpec, LLC",1,,
1,Jan 11 2023 2:09PM,254821,21,"NBTY Global, Inc.",1,,
2,Jan 11 2023 2:05PM,254819,21,"NBTY Global, Inc.",1,,
3,Jan 11 2023 1:56PM,254815,10,ISDIN Corporation,4,,
4,Jan 11 2023 1:55PM,254814,15,"Alliance Pharma, Inc.",53,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 2:22PM,254822,10,"ClearSpec, LLC",1.0,NaN,NaN
1,Jan 11 2023 2:09PM,254821,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jan 11 2023 2:05PM,254819,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jan 11 2023 1:56PM,254815,10,ISDIN Corporation,4.0,NaN,NaN
4,Jan 11 2023 1:55PM,254814,15,"Alliance Pharma, Inc.",53.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 2:22PM,254822,10,"ClearSpec, LLC",1.0,0.0,0.0
1,Jan 11 2023 2:09PM,254821,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jan 11 2023 2:05PM,254819,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jan 11 2023 1:56PM,254815,10,ISDIN Corporation,4.0,0.0,0.0
4,Jan 11 2023 1:55PM,254814,15,"Alliance Pharma, Inc.",53.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6369442,88.0,16.0,7.0
12,254793,0.0,1.0,0.0
15,254814,53.0,0.0,0.0
16,509491,1.0,1.0,0.0
19,1018957,34.0,9.0,1.0
21,3057547,12.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6369442,88.0,16.0,7.0
1,12,254793,0.0,1.0,0.0
2,15,254814,53.0,0.0,0.0
3,16,509491,1.0,1.0,0.0
4,19,1018957,34.0,9.0,1.0
5,21,3057547,12.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,88.0,16.0,7.0
1,12,0.0,1.0,0.0
2,15,53.0,0.0,0.0
3,16,1.0,1.0,0.0
4,19,34.0,9.0,1.0
5,21,12.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,88.0
1,12,Released,0.0
2,15,Released,53.0
3,16,Released,1.0
4,19,Released,34.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,7.0,0.0,0.0,0.0,1.0,0.0
Executing,16.0,1.0,0.0,1.0,9.0,0.0
Released,88.0,0.0,53.0,1.0,34.0,12.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,7.0,0.0,0.0,0.0,1.0,0.0
1,Executing,16.0,1.0,0.0,1.0,9.0,0.0
2,Released,88.0,0.0,53.0,1.0,34.0,12.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,7.0,0.0,0.0,0.0,1.0,0.0
1,Executing,16.0,1.0,0.0,1.0,9.0,0.0
2,Released,88.0,0.0,53.0,1.0,34.0,12.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()